### Universidad Nacional de Lujan - Bases de Datos Masivas (11088) - Cavasin Nicolas #143501
# TP05-01 - Árboles de decisión

### Ejercicio 4:
Se provee la base de datos de los pasajeros del famoso barco que se hundiera en su viaje inaugural (archivo titanic-en.csv) con los siguientes atributos y valores posibles: 
- Class {"1st","2nd","3rd","crew"}
- Age {"adult","child"} 
- Sex {"male","female"}
- Survived {"yes","no"}

Genere el árbol de clasificación, explore la solución dada y las posibles alternativas para obtener un nuevo árbol que clasifique “mejor”.
    

In [2]:
! rm titanic-en.csv
!wget https://raw.githubusercontent.com/bdm-unlu/2020/master/TPs/TP05/TP0501/titanic-en.csv

--2020-11-12 20:53:12--  https://raw.githubusercontent.com/bdm-unlu/2020/master/TPs/TP05/TP0501/titanic-en.csv
Loaded CA certificate &#39;/etc/ssl/certs/ca-certificates.crt&#39;
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.216.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.216.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42177 (41K) [text/plain]
Saving to: &#39;titanic-en.csv&#39;

titanic-en.csv      100%[===================&gt;]  41.19K  --.-KB/s    in 0.03s   

2020-11-12 20:53:12 (1.28 MB/s) - &#39;titanic-en.csv&#39; saved [42177/42177]



In [3]:
# Importo pandas para lectura
import pandas as pd

# Leo el archivo
titanic = pd.read_csv('titanic-en.csv')

# Muestro sus dimensiones
print(f'Cantidad de tuplas: {titanic.shape[0]}\n')
print(f'Cantidad de columnas: {titanic.shape[1]}\n')

titanic.head()

Cantidad de tuplas: 2201

Cantidad de columnas: 4



,class,age,sex,survived
0,1st,adult,male,yes
1,1st,adult,male,yes
2,1st,adult,male,yes
3,1st,adult,male,yes
4,1st,adult,male,yes


Se observa que todas las columnas son de tipo *String* (todas categóricas a excepción de la columna *class*) por lo tanto todas deben ser transformadas para poder luego crear el modelo de predicción. Para ello, se utilizará el método *LabelEncoder* -para las columnas categóricas- y adicionalmente el método *OneHotEncoder* (para la columna *class*) dando como resultado una columna binaria por cada valor diferente, una suerte de bitmap de valores (ver referencias).

In [4]:
# Importo el metodo
from sklearn.compose import ColumnTransformer

# Importo el encoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Instancio los encoders
le = LabelEncoder()
ohe = OneHotEncoder(sparse=True)

# Copio el dataset
copia = titanic.copy()

# Defino el target y lo saco del dataset a transformar
target = copia.pop('survived')
target_name = target.name

# Elimino la columna class del dataframe
col_class = copia.pop('class')

# Aplico LabelEncoder a todas las columnas
for col in copia.columns:
    copia[col] = le.fit_transform(copia[col])

## Inicio la transformacion de la columna class
# LabelEncoder - Analizo los dif valores de las clases de class 
le.fit(col_class)

# Labelencoder - Transformo las clases de class y le doy forma de columna 
class_as_int = le.transform(le.classes_).reshape(4, 1)

# OneHotEncoder - Creo una columna binaria por cada valor 
# diferente que el LabelEncoder haya generado sobre las classes
ohe.fit(class_as_int)

# Me guardo el numero de tuplas del dataframe
rows = copia.shape[0]

# LabelEncoder - Transformo los datos de str a integer y 
# le doy forma de columna
class_le = le.transform(col_class).reshape(rows, 1)

# OneHotEncoder - Creo una columna binaria por cada valor 
# diferente que el LabelEncoder haya generado sobre los datos 
class_ohe = pd.DataFrame(ohe.transform(class_le).toarray())

# Agrego el resultado de la transformacion al dataframe
titanic_transf = pd.concat([class_ohe, copia], axis=1)

# Me guardo el nombre de las features
features_names = list(le.classes_)
features_names.append('age')
features_names.append('sex')

# Defino las features
features = titanic_transf

# Muestro el resultado final
titanic_transf.head()

,0,1,2,3,age,sex
0,1.0,0.0,0.0,0.0,0,1
1,1.0,0.0,0.0,0.0,0,1
2,1.0,0.0,0.0,0.0,0,1
3,1.0,0.0,0.0,0.0,0,1
4,1.0,0.0,0.0,0.0,0,1


Como se observa, el dataset ha sido completamente transformado a valores discretos.

Esta listo para ser procesado con el fin de crear el árbol de decisión que determine si -dado el sexo, la edad y la clase en la que viajaban- el pasajero sobrevive o no al hundimiento del barco.

In [6]:
# Importo la libreria que permite separar en training set y test set
from sklearn.model_selection import train_test_split as s

# Importo el arbol
from sklearn import tree

# Importo libreria para graficar
import graphviz

# Importo las metricas para testear el modelo
from sklearn import metrics

# Divido el dataset en 75-25 para training y test respectivamente
features_train, features_test, target_train, target_test = s(features, target, random_state=0, test_size=0.25)

# Instancio el arbol por entropia
t = tree.DecisionTreeClassifier(criterion='entropy')

# Lo entreno
t.fit(features_train, target_train)

# Almaceno el modelo en formato DOT
titanic_dot = tree.export_graphviz(t
                    , out_file=None
                    , feature_names=features_names
                    , class_names=target.name
                    , label='all'
                    , filled=True, rounded=True
                    , special_characters=True)

# Tambien lo guardo como PNG
graph = graphviz.Source(titanic_dot)
graph.format = 'png'
graph.render('titanic')

# Obtengo la prediccion con el set de prueba definido anteriormente
le.fit(titanic['survived'])
prediccion = t.predict(features_test)

# Convierto a array para poder imprimirlo + facil
target_test = pd.array(target_test)

print(f'Se muestran los primeros 10 resultados\n')

# Muestro prediccion vs original
header = '{:<4}{:<2}{:<15}{:<2}{:<11}{}'.format('#', '|', 'Predicción', '|', 'Original','|')
print(header)
print('='*35, end='')
sep = '|'
for i in range(10):
    print(f"\n{str(i).ljust(4, ' ')}{sep.ljust(2, ' ')}{prediccion[i].ljust(15, ' ')}{sep.ljust(2, ' ')}{target_test[i].ljust(11, ' ')}{sep}\n", end='-'*35)

# Veo qué tan acertado estuvo
print(f'\n\nPrecisión del modelo: {metrics.accuracy_score(target_test, prediccion)}', end='\n\n')

# Muestro un reporte de clasificación con diferentes métricas sobre cada feature 
print(f'Reporte de clasificación: \n{metrics.classification_report(target_test, prediccion)}')



Se muestran los primeros 10 resultados

#   | Predicción     | Original   |
0   | no             | yes        |
-----------------------------------
1   | yes            | yes        |
-----------------------------------
2   | no             | no         |
-----------------------------------
3   | no             | yes        |
-----------------------------------
4   | no             | no         |
-----------------------------------
5   | no             | no         |
-----------------------------------
6   | no             | yes        |
-----------------------------------
7   | no             | no         |
-----------------------------------
8   | no             | no         |
-----------------------------------
9   | no             | yes        |
-----------------------------------

Precisión del modelo: 0.7858439201451906

Reporte de clasificación: 
              precision    recall  f1-score   support

          no       0.76      0.98      0.86       365
         yes       0.93  

Árbol de clasificación del dataset *titanic-en.csv*:

![Arbol](titanic.png)

***Observaciones:***
- El árbol resultante tiene una profundidad de 5 niveles.
- Tiene una precisión de clasificación del 78.5%.
- Falla más frecuentemente en la predicción de los quién no sobrevivirá. Es decir, es pesimista.
- La hoja con menos samples posee 1.

___


Para intentar mejorar su precisión, se intentarán las siguientes modificaciones:
- Poda de niveles con máximo 2.
- Límite mínimo de samples por hoja en 200.
- Árbol que clasifique aplicando Gini.
- Modificación del porcentaje de splitting entre sets de test y training (60%-40%) junto con aleatorización de los valores que cada uno utiliza (``random_state=33``) .

In [5]:
# Divido el dataset en 75-25 para training y test respectivamente
features_train, features_test, target_train, target_test = s(features, target, random_state=0, test_size=0.25)

# Instancio el arbol por entropia
t2 = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2)

# Lo entreno
t2.fit(features_train, target_train)

# Almaceno el modelo en formato DOT
titanic_dot = tree.export_graphviz(t2
                    , out_file=None
                    , feature_names=features_names
                    , class_names=target_name
                    , label='all'
                    , filled=True, rounded=True
                    , special_characters=True)

# Tambien lo guardo como PNG
graph = graphviz.Source(titanic_dot)
graph.format = 'png'
graph.render('titanic_max_level')

# Obtengo la prediccion con el set de prueba definido anteriormente
prediccion2 = t2.predict(features_test)

# Veo qué tan acertado estuvo
print(f'\n\nPrecisión del modelo: {metrics.accuracy_score(target_test, prediccion2)}', end='\n\n')

# Muestro un reporte de clasificación con diferentes métricas sobre cada feature 
print(f'Reporte de clasificación: \n{metrics.classification_report(target_test, prediccion2)}')





Precisión del modelo: 0.778584392014519

Reporte de clasificación: 
              precision    recall  f1-score   support

          no       0.76      0.98      0.85       365
         yes       0.92      0.38      0.53       186

    accuracy                           0.78       551
   macro avg       0.84      0.68      0.69       551
weighted avg       0.81      0.78      0.75       551



Árbol número 2: cantidad máxima de niveles = 2.

![Arbol](titanic_max_level.png)

In [6]:
# Divido el dataset en 75-25 para training y test respectivamente
features_train, features_test, target_train, target_test = s(features, target, random_state=0, test_size=0.25)

# Instancio el arbol por entropia
t3 = tree.DecisionTreeClassifier(criterion='entropy', min_samples_leaf=200)

# Lo entreno
t3.fit(features_train, target_train)

# Almaceno el modelo en formato DOT
titanic_dot = tree.export_graphviz(t3
                    , out_file=None
                    , feature_names=features_names
                    , class_names=target_name
                    , label='all'
                    , filled=True, rounded=True
                    , special_characters=True)

# Tambien lo guardo como PNG
graph = graphviz.Source(titanic_dot)
graph.format = 'png'
graph.render('titanic_min_samples')

# Obtengo la prediccion con el set de prueba definido anteriormente
prediccion3 = t3.predict(features_test)

# Veo qué tan acertado estuvo
print(f'\n\nPrecisión del modelo: {metrics.accuracy_score(target_test, prediccion3)}', end='\n\n')

# Muestro un reporte de clasificación con diferentes métricas sobre cada feature 
print(f'Reporte de clasificación: \n{metrics.classification_report(target_test, prediccion3)}')





Precisión del modelo: 0.7658802177858439

Reporte de clasificación: 
              precision    recall  f1-score   support

          no       0.78      0.91      0.84       365
         yes       0.73      0.49      0.59       186

    accuracy                           0.77       551
   macro avg       0.75      0.70      0.71       551
weighted avg       0.76      0.77      0.75       551



Árbol número 3: cantidad mínima de samples por hoja = 200.

![Arbol](titanic_min_samples.png)

In [7]:
# Divido el dataset en 75-25 para training y test respectivamente
features_train, features_test, target_train, target_test = s(features, target, random_state=0, test_size=0.25)

# Instancio el arbol por entropia
t4 = tree.DecisionTreeClassifier(criterion='gini')

# Lo entreno
t4.fit(features_train, target_train)

# Almaceno el modelo en formato DOT
titanic_dot = tree.export_graphviz(t4
                    , out_file=None
                    , feature_names=features_names
                    , class_names=target_name
                    , label='all'
                    , filled=True, rounded=True
                    , special_characters=True)

# Tambien lo guardo como PNG
graph = graphviz.Source(titanic_dot)
graph.format = 'png'
graph.render('titanic_gini')

# Obtengo la prediccion con el set de prueba definido anteriormente
prediccion4 = t4.predict(features_test)

# Veo qué tan acertado estuvo
print(f'\n\nPrecisión del modelo: {metrics.accuracy_score(target_test, prediccion4)}', end='\n\n')

# Muestro un reporte de clasificación con diferentes métricas sobre cada feature 
print(f'Reporte de clasificación: \n{metrics.classification_report(target_test, prediccion4)}')





Precisión del modelo: 0.7858439201451906

Reporte de clasificación: 
              precision    recall  f1-score   support

          no       0.76      0.98      0.86       365
         yes       0.93      0.40      0.56       186

    accuracy                           0.79       551
   macro avg       0.84      0.69      0.71       551
weighted avg       0.82      0.79      0.76       551



Árbol número 4: clasificación aplicando el criterio *Gini*.

![Arbol](titanic_gini.png)

In [8]:
# Divido el dataset en 60-40 para training y test respectivamente
features_train, features_test, target_train, target_test = s(features, target, random_state=33, test_size=0.4)

# Instancio el arbol por entropia
t5 = tree.DecisionTreeClassifier(criterion='entropy')

# Lo entreno
t5.fit(features_train, target_train)

# Almaceno el modelo en formato DOT
titanic_dot = tree.export_graphviz(t5
                    , out_file=None
                    , feature_names=features_names
                    , class_names=target_name
                    , label='all'
                    , filled=True, rounded=True
                    , special_characters=True)

# Tambien lo guardo como PNG
graph = graphviz.Source(titanic_dot)
graph.format = 'png'
graph.render('titanic_split')

# Obtengo la prediccion con el set de prueba definido anteriormente
prediccion5 = t5.predict(features_test)

# Veo qué tan acertado estuvo
print(f'\n\nPrecisión del modelo: {metrics.accuracy_score(target_test, prediccion5)}', end='\n\n')

# Muestro un reporte de clasificación con diferentes métricas sobre cada feature 
print(f'Reporte de clasificación: \n{metrics.classification_report(target_test, prediccion5)}')



Precisión del modelo: 0.8047673098751419

Reporte de clasificación: 
              precision    recall  f1-score   support

          no       0.78      0.99      0.87       595
         yes       0.94      0.43      0.59       286

    accuracy                           0.80       881
   macro avg       0.86      0.71      0.73       881
weighted avg       0.83      0.80      0.78       881



Árbol número 5: set de training del 60% del dataset y una aleatorización de selección con la semilla iniciada en el número 33.

![Arbol](titanic_split.png)

## Conclusiones:
- El árbol que mayor precisión alcanzó fué el último, donde se disminuyó el set de training al 60% del dataset y se le dió un valor de aleatorización a la selección de registros con la semilla iniciada en el número 33. Este modelo llega al 80.4% de precisión.

- El modelo menos preciso (76.5%) es el que fué instanciado con una cantidad mínima de 200 samples por hoja. Su retracción de niveles impide una buena clasificación.

- Lo mismo sucedió con el modelo cuyo máximo límite de niveles fué 2, la poda es excesiva y se terminan eliminando features (i.e.: *crew*).

- Por último, el modelo que clasifica utilizando el criterio *Gini* obtuvo la misma precisión que el modelo original: 78.5%.

### Referencias:

- Utilización del método [OneHotencoder()][0].

[0]:https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html?highlight=one%20hot%20encoder#sklearn.preprocessing.OneHotEncoder